In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join, isdir


In [2]:
data_folder = '../data/'
#test_folder = data_folder + 'test/'
train_folder = data_folder + 'train/'
list_train_paths = [join(train_folder,f+'/train.txt') for f in listdir(train_folder)]


In [3]:
list_train_paths

['../data/train/sst2/train.txt',
 '../data/train/trec/train.txt',
 '../data/train/cr/train.txt',
 '../data/train/cardio/train.txt',
 '../data/train/subj/train.txt',
 '../data/train/pc/train.txt',
 '../data/train/kaggle_med/train.txt']

In [4]:

def check_format_correction(path):
    with open(path,'r') as f:    
        lines = f.readlines()    
        for line in lines:
            if '\t' not in line:
                try:
                    line.replace(' ', '\t', 1)

                except:
                    print('error')
                    return False
    return True


def load_data(path):
    """
    Loads data from a txt file.
    """
    # check file format
    if path.endswith('.txt'):
        df = pd.read_csv(path, sep='|', header=None, names=['text'])
        try:
            df['class'] = df['text'].apply(lambda x: x.split('\t')[0])
            df['text'] = df['text'].apply(lambda x: x.split('\t')[1])
        except:
            df['class'] = df['text'].apply(lambda x: x.split(' ',1)[0])
            df['text'] = df['text'].apply(lambda x: x.split(' ',1)[1])

        df = df[['class', 'text']]
        return df
    else:
        raise ValueError('File format not supported.')


def generate_sample(path,sample_size):    
    if check_format_correction(path):
        output_path = path.replace('.txt','100_sample.txt')
        df = load_data(path)
        number_of_classes = df['class'].nunique()
        lowest_class_count = df['class'].value_counts().min()
        samples_per_class = int(sample_size/number_of_classes)
        if samples_per_class > lowest_class_count:
            samples_per_class = lowest_class_count
        new_df = df.groupby('class').apply(lambda x: x.sample(samples_per_class))
        new_df = new_df.sample(frac=1).reset_index(drop=True)   
        np.savetxt(output_path, new_df.values,fmt='%s',delimiter='\t')
        sample_name = output_path.split('/')[-2]
        print(f'sample {sample_name} with {len(new_df)} sentences is saved to: ',output_path)
    else:
        print('check the format of the file: ',path)

In [5]:
list_test_paths = ['../data/test/pc/test.txt','../data/test/cr/test.txt','../data/test/subj/test.txt']
sample_size = 30                 
for path in tqdm(list_test_paths):
    generate_sample(path,sample_size)


100%|██████████| 3/3 [00:00<00:00, 140.50it/s]

sample pc with 30 sentences is saved to:  ../data/test/pc/test100_sample.txt
sample cr with 30 sentences is saved to:  ../data/test/cr/test100_sample.txt
sample subj with 30 sentences is saved to:  ../data/test/subj/test100_sample.txt


In [93]:
train_folder = '../eda_code/txt_for_test/train/'
list_train_paths = [join(train_folder,f+'/train1000_sample.txt') for f in listdir(train_folder)]
list_train_paths

['../eda_code/txt_for_test/train/sst2/train1000_sample.txt',
 '../eda_code/txt_for_test/train/trec/train1000_sample.txt',
 '../eda_code/txt_for_test/train/cr/train1000_sample.txt',
 '../eda_code/txt_for_test/train/cardio/train1000_sample.txt',
 '../eda_code/txt_for_test/train/subj/train1000_sample.txt',
 '../eda_code/txt_for_test/train/pc/train1000_sample.txt',
 '../eda_code/txt_for_test/train/kaggle_med/train1000_sample.txt']

In [94]:
for i in list_train_paths:
    check_format_correction(i)

In [95]:
path  = '../eda_code/txt_for_test/train/pc/train1000_sample.txt'
df = pd.read_csv(path, sep='|', header=None, names=['text'])

df['class'] = df['text'].apply(lambda x: x.split('\t')[0])
df['text'] = df['text'].apply(lambda x: x.split('\t')[1])

IndexError: list index out of range